In [ ]:
import os
import json
from openai import AzureOpenAI

# --- Custom Function ---
def get_weather(location):
    # You can expand this logic to call a real API or customize based on location
    return {
        "location": location,
        "temperature": "70 degrees",
        "description": f"The weather in {location} is sunny and warm."
    }

# --- Azure OpenAI Client ---
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2025-03-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# --- First Call with Tool Definition ---
response = client.responses.create(
    model="gpt-4o",
    tools=[
        {
            "type": "function",
            "name": "get_weather",
            "description": "Get the weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                },
                "required": ["location"],
            },
        }
    ],
    input=[{"role": "user", "content": "What's the weather in San Francisco?"}],
)

print(response.model_dump_json(indent=2))

# --- Tool Call Handling ---
input = []
for output in response.output:
    if output.type == "function_call":
        if output.name == "get_weather":
            args = json.loads(output.arguments)
            result = get_weather(**args)  # Call your custom function
            input.append({
                "type": "function_call_output",
                "call_id": output.call_id,
                "output": json.dumps(result),
            })
        else:
            raise ValueError(f"Unknown function call: {output.name}")

# --- Follow-Up Call with Tool Output ---
second_response = client.responses.create(
    model="gpt-4o",
    previous_response_id=response.id,
    input=input
)

print(second_response.model_dump_json(indent=2))
